# Coding Confirmation to 일람표 Converter

In [8]:
import utility

uploaded = utility.prompt_upload()

Upload Coding Confirmation (.xlsx)


FileUpload(value={}, accept='*.xlsx', description='Upload')

In [7]:
final = utility.control_process(uploaded[0], uploaded[1], option=0)
final

Note: 일람표 포맷 선택 되었습니다. 포맷 변경은 option=1로 가능합니다.
File submitted...
****************************************
Excel column summary
    Expectedness : ['Expected AE']
    중대성 : ['아니오' '예']
    ADR 여부 : ['ADR']
    자료원 : ['자발보고']
****************************************
Excel MedDRA (SOC, PT) summary
    SOC
   * Gastrointestinal disorders (각종 위장관 장애)
   * Nervous system disorders (각종 신경계 장애)
    PT
   * Constipation (변비)
   * Paraesthesia (지각 이상)
****************************************
Check if 차수 is correct:

['1-2차']



	모드를 숫자로 선택 후 'Enter'키로 이동하세요:
	1) 제목 삭제 모드 (모든 매치 제거) 
	2) 제목 수정 모드 (오타인 경우) 
	3) 확인. 다음 
	Number:  3


중대성                                     이상사례종류     중대한           중대하지 않은  \
차수                                                1-2차              1-2차   
ADR 여부                                         이상사례, 건 약물이상반응, 건 이상사례, 건   
0       Gastrointestinal disorders (각종 위장관 장애)       0         0       1   
1                            Constipation (변비)       0         0       1   
2         Nervous system disorders (각종 신경계 장애)       1         1       1   
3                         Paraesthesia (지각 이상)       1         1       1   

중대성                  허가사항반영여부   수집원  
차수                                   
ADR 여부 약물이상반응, 건                     
0              1                     
1              1  Expected AE  자발보고  
2              1                     
3              1  Expected AE  자발보고

In [10]:
final.to_excel("파일 이름.xlsx")